<a href="https://colab.research.google.com/github/JaroslavHolecek/Teaching/blob/master/JupyterNotebook/SQL/Vztahy_11_1N_MN_zadani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relace (vztahy) mezi tabulkami
Jedním z cílů kvalitního návrhu databáze je odstranit redundanci dat (duplicitní - opakující se data).

Jednoduše "každá informace existuje v databázi pouze jednou".

Dosáhneme toho tak, že data rozdělíme do více tabulek s různými entitami (objekty), aby byla každá entita reprezentována pouze jednou.

Potom je nutné SŘBD poskytnout způsob, jak rozdělené informace opět spojit a také hlídat, že jsou data správně (jsou konzistentí).

K tomu využíváme vlastní a cizí klíče. Chcete-li však tento krok provést správně, je nutné pochopit vztahy (relace) mezi tabulkami a tyto relace potom v databázi správně zaznamenat.

## Vztah 1:1
Jedná se o spojení, kdy jedné položce v první tabulce odpovídá jedna položka v druhé tabulce. Nebo jedné položce v tabulce odpovídá jdna konkrétní informace. 

Příkladem může být tabulka Občan a tabulka Trvalé bydliště, kdy jednomu záznamu v tabulce Občan odpovídá jeden záznam v tabulce Trvalé bydliště a noapak jednomu záznamu v tabulce Trvalé bydliště odpovídá jeden záznam v tabulce Občan.

(Předpokládejme, že na jedné adrese může mít pouze jeden človšk trvalé bydliště)

Nebo také informace "jméno" v tabulce Občan - každý občan má jedno jméno (více se jich do buňky v tabulce nevejde) a také každé jméno patří pouze jednomu občanovi (řádku v tabulce Občan)

<img src="https://raw.githubusercontent.com/JaroslavHolecek/Teaching/master/JupyterNotebook/SQL/Images/JupiterNotebook_11_vztah.png" alt="Příklad vztahu 1:1">

SQL kód pro vztah 1:1 by mohl vypadat následovně:

In [ ]:
!pip install mysql.connector

In [9]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)

mycursor = mydb.cursor()


try:
  mycursor.execute("""DROP TABLE Obcan""")
except:
  print("Tabulka Obcan zřejmě neexistovala")
try:
  mycursor.execute("""DROP TABLE TrvaleBydliste""")
except:
  print("Tabulka TrvaleBydliste zřejmě neexistovala")

# Mějme tabulku Trvalé bydliště
mycursor.execute("""CREATE TABLE TrvaleBydliste(
    id int PRIMARY KEY AUTO_INCREMENT,
    trvale_bydliste char(100) UNIQUE NOT NULL
)""") 

# a mějme tabulku Občan
mycursor.execute("""CREATE TABLE Obcan(
    id int PRIMARY KEY AUTO_INCREMENT,
    jmeno text,
    prijmeni text,
    trvale_bydliste int UNIQUE, /* UNIQUE zabrání přiřazení stejného trv. bydlište více občanům */
    FOREIGN KEY (trvale_bydliste) REFERENCES TrvaleBydliste(id)
)""")

# Použitím cizího klíče jsme vytvořili mezi dvěma tabulkami vztah 1:1 -> To, že nemůžeme více občanům přiřadit stejné trvalé bydliště je docíleno slovem UNIQUE v tabulce Obcan.
# V tomto případě bychom mohli tabulky spojit do jedné - záleží na (našem) návrhu DB a předpokládaném použití .

# mycursor.execute("""DROP TABLE Obcan, TrvaleBydliste""")

mydb.commit()

## Vztah 1:N
Jedné položce (záznamu) v první tabulce odpovídá n-položek (více záznamů) v druhé tabulce. 

Jako příklad lze uvést tabulku Skladatel a tabulku Píseň, kde píseň může mít pouze jednoho skladatele, ale skladatel může mít n-písní.

<img src="https://raw.githubusercontent.com/JaroslavHolecek/Teaching/master/JupyterNotebook/SQL/Images/JupiterNotebook_1N_vztah.png" alt="Příklad vztahu 1:N">


Vztah 1:N by v SQL by mohl vypadat následovně

In [10]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Mějme tabulku Skladatel
mycursor.execute("""CREATE TABLE Skladatel(
    id int PRIMARY KEY AUTO_INCREMENT,
    jmeno text,
    prijmeni text
)""") 

# a mějme tabulku Píseň
mycursor.execute("""CREATE TABLE Pisen(
    id int PRIMARY KEY AUTO_INCREMENT,
    nazev_skladby text,
    autor int, 
    FOREIGN KEY (autor) REFERENCES Skladatel(id)
)""")

# Použitím cizího klíče jsme vytvořili mezi dvěma tabulkami vztah 1:N.
# SQL dotazy pro vytvoření 1:N vztahu se nijak kromě UNIQUE nijak neliší od dotazů pro vytvoření vztahu 1:1.
# Důležitý je však myšlenkový pochod (rozdíl) mězi těmito vztahy.

# Kdybychom chtěli tabulky Skladatel a Píseň spojit do jedné tabulky, tak by náš
# skladatel, už nesměl nic složit. Kdyby skladatel složil novou píseň, 
# tak bychom museli do tabulky přidat nový atribut a to pokaždé, 
# když skladatel složí něco nového.
# A zároveň bychom se tímto způsobem připravili o jednoduchý způsob jak vypsat všechny písně.

# Případně bychom museli ke každé písni zapisovat stále stejné údaje o skladateli -> data by se opakovala

# mycursor.execute("""DROP TABLE Pisen, Skladatel""")

mydb.commit()

## Vztah M:N
m-položkám (každému z více záznamů) v první tabulce může odpovídat n-položek (více záznamů) v druhé tabulce a naopak každému z více záznamů v druhé tabulce může odpovídat více záznamů v první tabulce.

Např. tabulka Píseň a tabulka Zpěvák. 

Jeden zpěvák může zpívat více písní a zároveň jednu píseň může zpívat více zpěváků.

<img src="https://raw.githubusercontent.com/JaroslavHolecek/Teaching/master/JupyterNotebook/SQL/Images/JupiterNotebook_MN_vztah.png" alt="Příklad vztahu M:N">

Abychom mohli tento vztah zachytot v databázi, musíme vytvoři třetí pomocnou "mezi" tabulku. Tuto tabulku někdy do ER-diagramu zakreslujeme, někdy ji tam nenajdete - záleží mimo volby návrháře také zda v této mezi-tabulce chceme uchovávat ještě nějaké přídavné informace (např. kdy zpěvák píseň nazpíval a pod.)

Kód pro vytvoření M:N vztahů by mohl vypadat následovně:

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host = "dbs.spskladno.cz"
    ,user = "rekne_ucitel"
    ,password = "rekne_ucitel"
    ,database = "zavisi_na_useru"
)
mycursor = mydb.cursor()

# Mějme tabulku Zpevak
mycursor.execute("""CREATE TABLE Zpevak(
    id int PRIMARY KEY AUTO_INCREMENT,
    jmeno text,
    prijmeni text
)""") 

# Mějme tabulku Píseň
mycursor.execute("""CREATE TABLE Pisen(
    id int PRIMARY KEY AUTO_INCREMENT,
    nazev_skladby text
)""")

# A mějme propojivací tabulku, protože M:N vyžaduje 3 tabulky.
mycursor.execute("""CREATE TABLE KdoCoNazpival(
    zpevak int NOT NULL,
    pisen int NOT NULL,
    FOREIGN KEY (zpevak) REFERENCES Zpevak(id),
    FOREIGN KEY (pisen) REFERENCES Pisen(id)
)""")

mydb.commit()

Následující obrázek znázorňuje jak může vypadat databáze s použitím 1:N a M:N vztahů.

Všimněte si, že v tabulkách jsou zapsané i "provazující" sloupečky.

 <img src="https://raw.githubusercontent.com/JaroslavHolecek/Teaching/master/JupyterNotebook/SQL/Images/JupiterNotebook_PrikladDatabaze.png" alt="Příklad vztahů v databázi">

## Cvičení
Dnešní cvičení bude pouze na zamyšlení.

Zamyslete se nad tím, jakým vztahem realizujeme následující tabulky a kde bude vlastní a cizí klíč

### Cvičení 1:
Mazlíček:
* id
* jake_zvire
* jmeno

Majitel:
* id
* jmeno 
* prijmeni 
* mazlicek

Vztah: 

Vlastní klíč: 

Cizí klíč: 

### Cvičení 2:
Pracovník:
* id 
* jmeno
* prijmeni

Služební Auto:
* id
* znacka
* spz
* ridic

Vztah: 

Vlastní klíč: 

Cizí klíč:

### Cvičení 3:
Učitel:
* id 
* jmeno
* prijmeni
* katedra

Předmět:
* id
* nazev

Vztah: 

Vlastní klíč: 

Cizí klíč: 
